<font color='red'> Penalty kick application based on 'Professionals Play Minimax' by Ignacio Palacios-Huerta (2003). </font>

### Imports

In [ ]:
from bokeh.io import show, output_notebook, save, output_file
from bokeh.models import Plot, TapTool, ColumnDataSource, LabelSet, StaticLayoutProvider, Circle, MultiLine
from bokeh.models.widgets import Div
from bokeh.models.graphs import NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.plotting import figure
from bokeh.models.renderers import GraphRenderer, GlyphRenderer
from bokeh.layouts import layout, row, column, gridplot
from bokeh.models import CustomJS, Button, Slider, DataTable, ColumnDataSource, TableColumn, Rect, Dropdown
from bokeh.models.glyphs import Text
from bokeh.events import ButtonClick

In [ ]:
output_notebook()

### Define Figure and Sprites

In [ ]:
p = figure(tools="", 
           toolbar_location=None, 
           title='FIFA 2020 Penalty Simulator',
           plot_width=600, plot_height=480, 
           x_range=(0, 100), y_range=(0, 90))
p.title.text_font_size = '15pt'

# hide axes and gridlines
p.xaxis.visible = False
p.yaxis.visible = False
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None

# background color
p.background_fill_color = "green"

# goal posts and lines
p.multi_line([[24, 76, 78, 22, 24], [3,  12, 88, 97, 3 ], [34, 34, 66, 66]],
             [[63, 63, 47, 47, 63], [15, 63, 63, 15, 15], [63, 82, 82, 63]],
             color=["lightgreen", "lightgreen", "whitesmoke"],
             alpha=[1, 1, 1], line_width=4)

# striker box
p.quadratic(33, 15, 67, 15, 50, 2, color='lightgreen', line_width=4)

# goalie sprite 
goalie_head = Circle(x=50, y=69, fill_color='red', line_width=2, size=17)
goalie_body = Rect(x=50, y=65, width=3, height=4, angle=0, fill_color='red', line_width=2)
p.add_glyph(goalie_head)
p.add_glyph(goalie_body)

# ball 
ball = Circle(x=50, y=13, fill_color='whitesmoke', line_width=2, size=17)
p.add_glyph(ball);

# striker sprite 
p.add_glyph(Circle(x=50, y=16, fill_color='lightblue', line_width=2, size=21)) # head
p.add_glyph(Rect(x=50, y=11, width=4, height=6, angle=0, fill_color='lightblue', line_width=2)); #body

### Define Labels

In [ ]:
scr_text = ColumnDataSource({'x': [2, 70, 2, 14, 14],
                             'y': [86, 86, 5, 40, 32],
                             'text': ['Rounds played: 0',
                                      'Total score: 0',
                                      '',
                                      '',
                                      '']})

labels = Text(x="x", y="y", text='text', text_color="whitesmoke",
              text_font_size= '15pt', x_offset=0, y_offset=+9,
              text_baseline='ideographic', text_align='left')

p.add_glyph(scr_text, labels);

### Define State Variables as Divs

In [ ]:
nround = Div(text='0', visible=False) # total number of rounds completed
score = Div(text='0', visible=False) # current score
freq_left_foot = Div(text='0,0,0', visible=False) # frequency of left, middle, and right kicks (left-footed)
freq_right_foot = Div(text='0,0,0', visible=False) # frequency of left, middle, and right kicks (right-footed)
kicker_foot = Div(text='', visible=False) # current footedness of the kicker
kicker_kick = Div(text='', visible=False) # current direction the kicker will kick

iterations_to_run = Div(text='50', visible=False) # Amount of iterations for the game to run

strategy_to_use = Div(text='Not Set', visible=False)

LL_chance = 1/6
LM_chance = 1/6
LR_chance = 1/6
RL_chance = 1/6
RM_chance = 1/6
RR_chance = 1/6

### Define Buttons

In [ ]:
# start and next buttons
b_start = Button(label="Begin", button_type="success", sizing_mode= 'scale_width', width_policy='fit')
b_next = Button(label="Next round", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=True)

# right and left footed buttons
b_fl = Button(label="Left-Footed", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=True)
b_fr = Button(label="Right-Footed", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=True)

# right, middle, and left kick buttons
bl = Button(label="Left", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)
bm = Button(label="Middle", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)
br = Button(label="Right", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)

# shoot
b_shoot = Button(label="SHOOT!", button_type="success", sizing_mode= 'scale_width', width_policy='fit', disabled=False, visible=False)

# automate buttons
b_automate = Button(label="Automate", button_type="success", sizing_mode='scale_width', width_policy='fit')
b_start_automate = Button(label="Start", button_type="success", sizing_mode='scale_width', width_policy='fit', disabled=False, visible=False)

#automate_next_round_button
b_auto_next=Button(label="Next", button_type="success", sizing_mode='scale_width', width_policy='fit', disabled=False, visible=False)

# aim direction chance weights
LL_aim_slider=Slider(start=0, end=1, value=1/6, step=0.01, title="LL Aim Weight", disabled=False, visible=False)
LM_aim_slider=Slider(start=0, end=1, value=1/6, step=0.01, title="LM Aim Weight", disabled=False, visible=False)
LR_aim_slider=Slider(start=0, end=1, value=1/6, step=0.01, title="LR Aim Weight", disabled=False, visible=False)
RL_aim_slider=Slider(start=0, end=1, value=1/6, step=0.01, title="RL Aim Weight", disabled=False, visible=False)
RM_aim_slider=Slider(start=0, end=1, value=1/6, step=0.01, title="RM Aim Weight", disabled=False, visible=False)
RR_aim_slider=Slider(start=0, end=1, value=1/6, step=0.01, title="RR Aim Weight", disabled=False, visible=False)

#Iterations To Run Slider
iterations_slider=Slider(start=10, end=500, value=50, step=1, title="Iterations To Run", disabled=False, visible=False)

#CPU To use dropdown
menu=[("Fictitious_Play", "Fictitious_Play"), ("Mixed_Strategy", "Mixed_Strategy")]
strategy_dropdown = Dropdown(label="CPU strategy to Use", menu=menu, button_type="warning", disabled=False, visible=False)

### Define Automation Distribution Tracking Table

In [ ]:
#Make Automation Distribution Tracking Table
distribution_data=dict(footedness=["Left","Left", "Left", "Right", "Right", "Right"], 
                       aim_direction=["Left", "Middle", "Right","Left", "Middle", "Right"],
                       freq=[0,0,0,0,0,0], decisions=[0,0,0,0,0,0], 
                       goalie_perceived_risks=[0,0,0,0,0,0],
                       striker_score_chance=[0,0,0,0,0,0], 
                       striker_score_roll=[0,0,0,0,0,0])

automation_distribution_table_source=ColumnDataSource(distribution_data)

distribution_columns = [TableColumn(field="footedness", title="Striker Footedness"),
                        TableColumn(field="aim_direction", title="Striker Aim Direction"),
                        TableColumn(field="freq", title="Frequency"),
                        TableColumn(field="decisions", title="Goalie Decisions"),
                        TableColumn(field="goalie_perceived_risks", title="Goalie's Perceived Risks"),
                        TableColumn(field="striker_score_chance", title="Striker's Score Chance"),
                        TableColumn(field="striker_score_roll", title="Striker's Score Roll")]
automation_distribution_table = DataTable(source=automation_distribution_table_source, 
                                          columns=distribution_columns, width=400, height=280, 
                                          autosize_mode="force_fit", visible=False)

### Define Callbacks

In [ ]:
# next button
b_next_code = """
b_shoot.visible = false;

b_fr.visible = true;
b_fl.visible = true;
b_fr.disabled = false;
b_fl.disabled = false;

bl.visible = false;
bm.visible = false;
br.visible = false;

b_next.disabled = true;

txt.data['text'][2] = ''
txt.data['text'][3] = 'Choose a right- or left footed kicker!'
txt.change.emit()  

ball.x = 50
ball.y = 13
goalie_head.x = 50
goalie_body.x = 50
"""
b_next_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br,
                                  txt=scr_text, b_shoot=b_shoot, b_next=b_next, ball=ball,
                                  goalie_head=goalie_head, goalie_body=goalie_body),
                        code=b_next_code)
b_next.js_on_click(b_next_click)

In [ ]:
# start button
b_start_code = b_next_code + """
if (b_start.label == 'Begin') {
    b_start.label = 'Restart'
} else {
    txt.data['text'][0] = 'Rounds played: 0'
    txt.data['text'][1] = 'Total score: 0'
    txt.data['text'][2] = ''
    txt.data['text'][3] = ''
    txt.data['text'][4] = ''
    nround.text='0'
    score.text='0'
    freq_left_foot.text='0,0,0'
    freq_right_foot.text='0,0,0'
    kicker_foot.text=''
    kicker_kick.text=''
    txt.change.emit() 
    b_start.label = 'Begin'
}
"""
b_start_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br,txt=scr_text,
                                   kicker_kick=kicker_kick, kicker_foot=kicker_foot, freq_left_foot=freq_left_foot, 
                                   b_next=b_next,freq_right_foot=freq_right_foot, b_shoot=b_shoot, nround=nround, 
                                   score=score, ball=ball, goalie_head=goalie_head, goalie_body=goalie_body),
                         code=b_start_code)
b_start.js_on_click(b_start_click)

In [ ]:
# select the foot type of striker buttons
def b_f_click(foot):
    code = """
    b_fl.disabled = true;
    b_fr.disabled = true;
    b_fl.visible = false;
    b_fr.visible = false;
    bl.visible = true;
    bm.visible = true;
    br.visible = true;
    txt.data['text'][3] = 'Choose where to kick!'
    txt.data['text'][2] = '"""+foot+"""-footed kicker.'
    kicker_foot.text= '"""+foot+"""'
    txt.change.emit()
    """
    return CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br, txt=scr_text,
                              kicker_foot=kicker_foot),
                    code=code)
b_fl.js_on_click(b_f_click('Left'))
b_fr.js_on_click(b_f_click('Right'))

In [ ]:
# select where the kicker will kick buttons
def b_kick_click(kick):
    code = """
    b_shoot.visible = true;
    bl.visible = false;
    bm.visible = false;
    br.visible = false;
    txt.data['text'][3] = ''
    var current = txt.data['text'][2]
    txt.data['text'][2] = current + ' Kicking """+kick+""".'
    kicker_kick.text= '"""+kick+"""'
    txt.change.emit()
    b_shoot.disabled = false;
    """
    return CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, bl=bl, bm=bm, br=br, txt=scr_text,
                              kicker_foot=kicker_foot, kicker_kick=kicker_kick, b_shoot=b_shoot),
                    code=code)

bl.js_on_click(b_kick_click('Left'))
bm.js_on_click(b_kick_click('Middle'))
br.js_on_click(b_kick_click('Right'))

In [ ]:
# shoot button
b_shoot_code = """
// Define probability matrix
var p = {'Right' : {'LeftLeft' : 0.55,
                    'LeftMiddle' : 0.65,
                    'LeftRight' : 0.93,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.95,
                    'RightMiddle' : 0.73,
                    'RightRight' : 0.70},
         'Left' :  {'LeftLeft' : 0.67,
                    'LeftMiddle' : 0.70,
                    'LeftRight' : 0.96,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.87,
                    'RightMiddle' : 0.65,
                    'RightRight' : 0.61}}

// Choose best action
var freq
if (kicker_foot.text == 'Right') {
    freq = freq_right_foot.text.split(',').map(Number);
} else {
    freq = freq_left_foot.text.split(',').map(Number);
}

var kicker_action = 'Left'
var expected = (freq[0]*p[kicker_foot.text]['LeftLeft'] + 
                freq[1]*p[kicker_foot.text]['MiddleLeft'] +
                freq[2]*p[kicker_foot.text]['RightLeft'])   
var actions = ["Left", "Middle", "Right"]
for (var i = 0; i < 3; i++) {
    var val = (freq[0]*p[kicker_foot.text]['Left'+actions[i]] + 
               freq[1]*p[kicker_foot.text]['Middle'+actions[i]] + 
               freq[2]*p[kicker_foot.text]['Right'+actions[i]])
    if (val < expected) {
        kicker_action = actions[i]
        expected = val
    }
}    

// Determine if goal
var goal = 1
if (Math.random() > p[kicker_foot.text][kicker_kick.text+kicker_action]) {
    goal = -1
}

// Animate ball and goalie
ball.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_kick.text]
ball.y = 63
goalie_head.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action]
goalie_body.x = {'Left' : 40, 'Middle' : 50, 'Right' : 60}[kicker_action]

// Add to frequency history
var dict = {'Left'  : 0,
            'Middle' : 1,
            'Right'   : 2}
if (kicker_foot.text == 'Right') {
    var freq = freq_right_foot.text.split(',')
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1
    freq_right_foot.text = freq.toString()
} else {
    var freq = freq_left_foot.text.split(',')
    freq[dict[kicker_kick.text]] = parseInt(freq[dict[kicker_kick.text]]) + 1
    freq_left_foot.text = freq.toString()
}

// Update text
var n = (parseInt(nround.text) + 1)
nround.text = n.toString()
txt.data['text'][0] = 'Rounds played: ' + n

if (goal == 1) {
    txt.data['text'][3] = 'GOAL!'
} else {
    txt.data['text'][3] = 'Blocked'
}

var s = parseInt(score.text) + goal
score.text = s.toString()
txt.data['text'][1] = 'Total score: ' + s

txt.change.emit()

// Allow starting next round
b_shoot.disabled = true;
b_next.disabled = false;
"""
b_shoot_click = CustomJS(args=dict(b_start=b_start, b_fl=b_fl, b_fr=b_fr, txt=scr_text, kicker_kick=kicker_kick,
                                   kicker_foot=kicker_foot, freq_left_foot=freq_left_foot, b_next=b_next,
                                   freq_right_foot=freq_right_foot, b_shoot=b_shoot, nround=nround, score=score,
                                   ball=ball, goalie_head=goalie_head, goalie_body=goalie_body),
                         code=b_shoot_code)
b_shoot.js_on_click(b_shoot_click)

In [ ]:
#Make Automation Decision Table

data=dict(footedness=["Left", "Left", "Left", "Right", "Right", "Right"], 
          aim_direction=["Left", "Middle", "Right", "Left", "Middle", "Right"],
          chances=[LL_chance, LM_chance, LR_chance, RL_chance, RM_chance, RR_chance])
automation_table_source=ColumnDataSource(data)
columns = [TableColumn(field="footedness", title="Striker Footedness"),
           TableColumn(field="aim_direction", title="Striker Aim Direction"),
           TableColumn(field="chances", title="Chance")]
automation_table = DataTable(source=automation_table_source, columns=columns, width=400, height=280, autosize_mode="force_fit", visible=False)

In [ ]:
# automate button
b_automate_code = """
b_automate.visible=false;

b_start.visible=false;
b_fl.visible=false;
b_fr.visible=false;
b_next.visible=false;

LL_aim_slider.visible=true;
LM_aim_slider.visible=true;
LR_aim_slider.visible=true;
RL_aim_slider.visible=true;
RM_aim_slider.visible=true;
RR_aim_slider.visible=true;

iterations_slider.visible=true;

strategy_dropdown.visible=true;

automation_table.visible=true;

txt.data['text'][2] = ''
txt.data['text'][3] = 'Choose a right- or left footed kicker!'
txt.change.emit()  
"""
b_automate_click = CustomJS(args=dict(b_automate=b_automate, b_start=b_start, b_fl=b_fl, b_fr=b_fr, 
                                      b_next=b_next, txt=scr_text,
                                      LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider,
                                      LR_aim_slider=LR_aim_slider, RL_aim_slider=RL_aim_slider,
                                      RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider,
                                      automation_table=automation_table, iterations_slider=iterations_slider,
                                      strategy_dropdown=strategy_dropdown), 
                            code=b_automate_code)
b_automate.js_on_click(b_automate_click)

In [ ]:
automate_start_code_initial_gui_display="""

b_start_automate.visible=false;
b_auto_next.visible=true;

LL_aim_slider.visible=false;
LM_aim_slider.visible=false;
LR_aim_slider.visible=false;
RL_aim_slider.visible=false;
RM_aim_slider.visible=false;
RR_aim_slider.visible=false;

iterations_slider.visible=false;

strategy_dropdown.visible=false;

automation_table.visible=false;
automation_distribution_table.visible=true;

"""
automate_loop_iteration_var_instantiations="""

// Define probability matrix
var p = {'Right' : {'LeftLeft' : 0.55,
                    'LeftMiddle' : 0.65,
                    'LeftRight' : 0.93,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.95,
                    'RightMiddle' : 0.73,
                    'RightRight' : 0.70},
         'Left' :  {'LeftLeft' : 0.67,
                    'LeftMiddle' : 0.70,
                   'LeftRight' : 0.96,
                    'MiddleLeft' : 0.74,
                    'MiddleMiddle' : 0.60,
                    'MiddleRight' : 0.72,
                    'RightLeft' : 0.87,
                    'RightMiddle' : 0.65,
                    'RightRight' : 0.61}}

//Obtain the Shot Aim chances from the column data source
    var chances_data=ChancesColumnDataSource.data
    var chances=chances_data['chances']
    var LL_chance=chances[0]
    var LM_chance=LL_chance+chances[1]
    var LR_chance=LM_chance+chances[2]
    var RL_chance=LR_chance+chances[3]
    var RM_chance=RL_chance+chances[4]

//Goalie Logic Decision Making Variables
    var chance_left=1/3;
    var chance_middle=1/3;
    var chance_right=1/3;

    var danger_goalie_left=0;
    var danger_goalie_middle=0;
    var danger_goalie_right=0;
    var selected_dict=0;

    var total_sample_rolls

    var goalie_action="None"

//Loop Randomness Variable
    var roll=0;

//Goalie Fictitious Learning Variables
    var selected_index=0;

    var dist_data=DistributionColumnDataSource.data
    var freq=dist_data['freq']
    var decisions=dist_data['decisions']

//Data Display Variables
    var perceived_risks=dist_data['goalie_perceived_risks']
    var scoring_chance=dist_data['striker_score_chance']
    var scoring_roll=dist_data['striker_score_roll']

//Striker Kick Choice Variables
    var kicker_foot='none'
    var kicker_kick='none'

//Scoring Variables
    var score_chance=0
    var goal=1

//Animation Variables
    var animation_positions = {'Left':[37, 43], 'Middle':[47,53], 'Right':[57, 63]}
    var bally=63
"""

automate_loop_iteration_display="""

txt.data['text'][0] = 'Rounds played: ' + n

if (goal == 1) {
    txt.data['text'][3] = 'GOAL!'
} 
else {
    txt.data['text'][3] = 'Blocked'
}
txt.data['text'][1] = 'Total score: ' + s

txt.change.emit()

DistributionColumnDataSource.change.emit()
"""

automate_loop_roll_kicker_action="""

//Handle Automated Selection
roll=Math.random();    
    
if(roll<=LL_chance){
    kicker_foot='Left'
    kicker_kick='Left'
}
else if(roll<=LM_chance){
    kicker_foot='Left'
    kicker_kick='Middle'
}
else if(roll<=LR_chance){
    kicker_foot='Left'
    kicker_kick='Right'
}
else if(roll<=RL_chance){
    kicker_foot='Right'
    kicker_kick='Left'
}
else if(roll<=RM_chance){
    kicker_foot='Right'
    kicker_kick='Middle'
}
else{
    kicker_foot='Right'
    kicker_kick='Right'
}

"""

automate_loop_handle_goalie_decision="""

//Handle Goalie Decision

if(kicker_foot=='Left'){
    total_sample_rolls=freq[0]+freq[1]+freq[2]
    selected_dict=p['Left']
    if(total_sample_rolls==0){
        chance_left=1/3
        chance_middle=1/3
        chance_right=1/3
    }
    else{
        chance_left=freq[0]/total_sample_rolls
        chance_middle=freq[1]/total_sample_rolls
        chance_right=freq[2]/total_sample_rolls
    }
}
else{
    total_sample_rolls=freq[3]+freq[4]+freq[5]
    selected_dict=p['Right']
    if(total_sample_rolls==0){
        chance_left=1/3
        chance_middle=1/3
        chance_right=1/3
    }
    else{
        chance_left=freq[3]/total_sample_rolls
        chance_middle=freq[4]/total_sample_rolls
        chance_right=freq[5]/total_sample_rolls
    }
}
danger_goalie_left=(chance_left*selected_dict['LeftLeft']+chance_middle*selected_dict['MiddleLeft']+chance_right*selected_dict['RightLeft']);
danger_goalie_middle=(chance_left*selected_dict['LeftMiddle']+chance_middle*selected_dict['MiddleMiddle']+chance_right*selected_dict['RightMiddle'])
danger_goalie_right=(chance_left*selected_dict['LeftRight']+chance_middle*selected_dict['MiddleRight']+chance_right*selected_dict['RightRight'])
    

if(danger_goalie_left<danger_goalie_middle){
    if(danger_goalie_left<danger_goalie_right){
        goalie_action="Left"
    }
    else if(danger_goalie_left==danger_goalie_right){
        roll=Math.random()
        if(roll<=0.5){
            goalie_action="Left"
        }
        else{
            goalie_action="Right"
        }
    }
    else{
        goalie_action="Right"
    }
}
else if (danger_goalie_left==danger_goalie_middle){
    roll=Math.random()
    if(roll<=0.5){
        goalie_action="Left"
    }
    else{
        goalie_action="Middle"
    }
}
else{
    if(danger_goalie_middle<danger_goalie_right){
        goalie_action="Middle"
    }
    else if(danger_goalie_middle==danger_goalie_right){
        roll=Math.random()
        if(roll<=0.5){
            goalie_action="Middle"
        }
        else{
            goalie_action="Right"
        }
    }
    else{
        goalie_action="Right"
    }
}

"""

automate_loop_handle_scoring = """

//Handle Score Chance:
    
roll=Math.random()
score_chance=p[kicker_foot][kicker_kick+goalie_action]
if(roll<=score_chance){
    goal=1
}
else{
    goal=-1
}
//Display Score Chance:

for(var i=0; i<=5; i++){
    scoring_chance[i]=0;
    scoring_roll[i]=0;
}

selected_index=0
if(kicker_foot=='Right'){
    selected_index+=3
}
if(kicker_kick=='Middle'){
    selected_index+=1
}
else if(kicker_kick=='Right'){
    selected_index+=2
}

scoring_chance[selected_index]=score_chance
scoring_roll[selected_index]=roll

// Update text
    
var n = (parseInt(nround.text) + 1)
if(n>=parseInt(iterations_to_run.text)){
    b_auto_next.visible=false;
}
nround.text = n.toString()
    
var s = parseInt(score.text) + goal
score.text = s.toString()

"""

automate_loop_animation="""

//Animate Scenario:

var animation_roll=0;
var animation_slot=0;
animation_roll=Math.random()

if(animation_roll<=0.5){
    animation_slot=1;
}

ball.x = animation_positions[kicker_kick][animation_slot]
ball.y = bally

if(goal==-1){
    if(goalie_action==kicker_kick){
        goalie_head.x = ball.x
        goalie_body.x = ball.x
    }
    else{
        if(kicker_kick=="Right"){
            ball.x=70
        }
        else if(kicker_kick=="Left"){
            ball.x=30
        }
        else{
            ball.x=[30,70][animation_slot]
        }
    }

}
else{
    if(goalie_action==kicker_kick){
        if(animation_slot==1){
            goalie_head.x=animation_positions[goalie_action][0]
            goalie_body.x=animation_positions[goalie_action][0]
        }
        else{
            goalie_head.x=animation_positions[goalie_action][1]
            goalie_body.x=animation_positions[goalie_action][1]
        }
    }else{
        if(animation_roll<=0.5){
            animation_slot=1;
        }else{
            animation_slot=0;
        }
        goalie_head.x=animation_positions[goalie_action][animation_slot]
        goalie_body.x=animation_positions[goalie_action][animation_slot]
    }
}

"""

automate_loop_update_fictitious_decision_tracking="""

//Update Goalie Frequency Tracking:
    
selected_index=0
if(kicker_foot=='Right'){
    selected_index+=3
}
if(kicker_kick=='Middle'){
    selected_index+=1
}
else if(kicker_kick=='Right'){
    selected_index+=2
}
    
freq[selected_index]+=1
    
    
//Update Goalie Decision Tracking:
    
selected_index=0
if(kicker_foot=='Right'){
    selected_index+=3
}
if(goalie_action=='Middle'){
    selected_index+=1
}
else if(goalie_action=='Right'){
    selected_index+=2
}
    
decisions[selected_index]+=1

//Update Goalie Perceived Risks:

selected_index=0
if(kicker_foot=='Right'){
    selected_index+=3
}
for(var i=0; i<=5; i++){
    perceived_risks[i]=0
}
perceived_risks[selected_index]=danger_goalie_left
perceived_risks[selected_index+1]=danger_goalie_middle
perceived_risks[selected_index+2]=danger_goalie_right
"""

automate_loop_iteration_main = (automate_loop_roll_kicker_action
                                + automate_loop_handle_goalie_decision
                                + automate_loop_handle_scoring
                                + automate_loop_animation
                                + automate_loop_update_fictitious_decision_tracking)

automate_loop_iteration=(automate_loop_iteration_var_instantiations
                         + automate_loop_iteration_main
                         + automate_loop_iteration_display)

b_automate_start_code = (automate_start_code_initial_gui_display
                         + automate_loop_iteration
                         )

b_start_automate_click = CustomJS(args=dict(b_automate=b_automate, b_start_automate=b_start_automate, b_auto_next=b_auto_next, 
                                            b_start=b_start, b_fl=b_fl, b_fr=b_fr, b_next=b_next, txt=scr_text,
                                            LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider,
                                            LR_aim_slider=LR_aim_slider, RL_aim_slider=RL_aim_slider,
                                            RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider,
                                            automation_table=automation_table, 
                                            automation_distribution_table=automation_distribution_table,
                                            ChancesColumnDataSource=automation_table_source, 
                                            DistributionColumnDataSource=automation_distribution_table_source,
                                            nround=nround, score=score, ball=ball, goalie_head=goalie_head, 
                                            goalie_body=goalie_body, iterations_slider=iterations_slider,
                                            iterations_to_run=iterations_to_run, strategy_dropdown=strategy_dropdown), 
                                  code=b_automate_start_code)
b_start_automate.js_on_click(b_start_automate_click)

In [ ]:
# b_auto_next callback
b_auto_next_code="""


"""+automate_loop_iteration

b_auto_next_click=CustomJS(args=dict(txt=scr_text, automation_table=automation_table, 
                                     automation_distribution_table=automation_distribution_table,
                                     ChancesColumnDataSource=automation_table_source, 
                                     DistributionColumnDataSource=automation_distribution_table_source,
                                     nround=nround, score=score, b_auto_next=b_auto_next, ball=ball,
                                     goalie_head=goalie_head, goalie_body=goalie_body, iterations_to_run=iterations_to_run),
                           code=b_auto_next_code)
b_auto_next.js_on_click(b_auto_next_click)

In [ ]:
aim_slider_callback_code="""
    var data=ColumnDataSourceToChange.data
    var chances=data['chances']
    
    var total=0
    total+=LL_aim_slider.value
    total+=LM_aim_slider.value
    total+=LR_aim_slider.value
    total+=RL_aim_slider.value
    total+=RM_aim_slider.value
    total+=RR_aim_slider.value
    
    chances[0]=LL_aim_slider.value/total
    chances[1]=LM_aim_slider.value/total
    chances[2]=LR_aim_slider.value/total
    chances[3]=RL_aim_slider.value/total
    chances[4]=RM_aim_slider.value/total
    chances[5]=RR_aim_slider.value/total
    
    ColumnDataSourceToChange.change.emit()
"""

LL_aim_callback = CustomJS(args=dict(ColumnDataSourceToChange=automation_table_source,
                                     LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider, LR_aim_slider=LR_aim_slider,
                                     RL_aim_slider=RL_aim_slider, RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider), 
                           code=aim_slider_callback_code)
LM_aim_callback = CustomJS(args=dict(ColumnDataSourceToChange=automation_table_source,
                                     LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider, LR_aim_slider=LR_aim_slider,
                                     RL_aim_slider=RL_aim_slider, RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider), 
                           code=aim_slider_callback_code)
LR_aim_callback = CustomJS(args=dict(ColumnDataSourceToChange=automation_table_source,
                                     LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider, LR_aim_slider=LR_aim_slider,
                                     RL_aim_slider=RL_aim_slider, RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider), 
                           code=aim_slider_callback_code)
RL_aim_callback = CustomJS(args=dict(ColumnDataSourceToChange=automation_table_source,
                                     LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider, LR_aim_slider=LR_aim_slider,
                                     RL_aim_slider=RL_aim_slider, RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider), 
                           code=aim_slider_callback_code)
RM_aim_callback = CustomJS(args=dict(ColumnDataSourceToChange=automation_table_source,
                                     LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider, LR_aim_slider=LR_aim_slider,
                                     RL_aim_slider=RL_aim_slider, RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider), 
                           code=aim_slider_callback_code)
RR_aim_callback = CustomJS(args=dict(ColumnDataSourceToChange=automation_table_source,
                                     LL_aim_slider=LL_aim_slider, LM_aim_slider=LM_aim_slider, LR_aim_slider=LR_aim_slider,
                                     RL_aim_slider=RL_aim_slider, RM_aim_slider=RM_aim_slider, RR_aim_slider=RR_aim_slider), 
                           code=aim_slider_callback_code)

LL_aim_slider.js_on_change('value', LL_aim_callback)
LM_aim_slider.js_on_change('value', LM_aim_callback)
LR_aim_slider.js_on_change('value', LR_aim_callback)
RL_aim_slider.js_on_change('value', RL_aim_callback)
RM_aim_slider.js_on_change('value', RM_aim_callback)
RR_aim_slider.js_on_change('value', RR_aim_callback)

In [ ]:
iterations_slider_code="""
    var iterations=parseInt(iterations_to_run.text);
    iterations=cb_obj.value;
    iterations_to_run.text=iterations.toString();
"""

iterations_slider_callback=CustomJS(args=dict(iterations_to_run=iterations_to_run), code=iterations_slider_code)
iterations_slider.js_on_change('value', iterations_slider_callback)

In [ ]:
strategy_dropdown_code="""
    strategy_dropdown.label=this.item;
    strategy_to_use.text=this.item;
    b_start_automate.visible=true;
"""

strategy_dropdown_callback=CustomJS(args=dict(strategy_dropdown=strategy_dropdown,
                                              b_start_automate=b_start_automate,
                                              strategy_to_use=strategy_to_use), 
                                    code=strategy_dropdown_code)
strategy_dropdown.js_on_event("menu_item_click", strategy_dropdown_callback)

### Format Layout and Show

In [ ]:
buttons_bot = row(b_start, nround, b_fl, b_fr, b_shoot, bl, bm, br,
                  score, freq_left_foot, freq_right_foot, kicker_foot, 
                  kicker_kick, b_next, max_width=600, sizing_mode='stretch_width')

automate_button = row(b_automate, iterations_slider,  b_auto_next, max_width=400, sizing_mode='stretch_width')
strategy_dropdown_row = row(strategy_dropdown, max_width=400, sizing_mode='stretch_width')
start_automate_row = row(b_start_automate, max_width=400, sizing_mode='stretch_width')
automate_LL_aim = row(LL_aim_slider, max_width=400, sizing_mode='stretch_width')
automate_LM_aim = row(LM_aim_slider, max_width=400, sizing_mode='stretch_width')
automate_LR_aim = row(LR_aim_slider, max_width=400, sizing_mode='stretch_width')
automate_RL_aim = row(RL_aim_slider, max_width=400, sizing_mode='stretch_width')
automate_RM_aim = row(RM_aim_slider, max_width=400, sizing_mode='stretch_width')
automate_RR_aim = row(RR_aim_slider, max_width=400, sizing_mode='stretch_width')

gui_column1=column(p, buttons_bot, max_width=600, sizing_mode='stretch_height')
gui_column2=column(automate_button, strategy_dropdown_row, start_automate_row,
                   automate_LL_aim, automate_LM_aim, automate_LR_aim,
                   automate_RL_aim, automate_RM_aim, automate_RR_aim,
                   automation_table, automation_distribution_table,
                   max_width=400, sizing_mode='stretch_height')
gui_row=row(gui_column1, gui_column2, max_width=1200, sizing_mode='stretch_height')
grid1 = gridplot([[gui_row]], plot_width=1000, plot_height=480)


show(grid1)

### Create HTML File

In [ ]:
output_file("penalty_kick_game.html")
save(grid);